In [11]:
# from serial import serial
import serial
import time

In [4]:
ser = serial.Serial('/dev/cu.usbserial-1410',9600)  # open serial port
print(ser.name)         # check which port was really used
# ser.write(b'sesu5,6,:nsetm3:l9:l10:l11:n40:255:255:255:nret')     # write a string
# ser.write(b'setm3:l9:l10:l11:');
# ser.write(b'40:255:255:255');
time.sleep(0.1)
ser.close()
print("done")


/dev/cu.usbserial-1410
done


In [116]:
ser = serial.Serial('/dev/cu.usbmodem1411',9600)  # open serial port
print(ser.name)         # check which port was really used

for i in range(10000000):
#     time.sleep(0.0001)
#     ser.write(b'0:')     # write a string
#     time.sleep(0.4)
    ser.write(b'req')
    read=ser.readline()
#     print(read)
    dist=int(str(read)[5:].split(':')[0])
    print(str(read)[5:].split(':')[0])
    string=""+str(min(dist*5,180))+":"+str(min(dist*3,255))+":"+str(255-min(dist*3,255))+":"+str(min(dist*3,255))+":"
    print(string)
    ser.write(string.encode('utf-8'))
    read=ser.readline()
#     time.sleep(0.01)
#     print(dist)

ser.close()

/dev/cu.usbmodem1411
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
3
15:9:246:9:
6
30:18:237:18:
8
40:24:231:24:
2379
180:255:0:255:
8
40:24:231:24:
7
35:21:234:21:
5
25:15:240:15:
5
25:15:240:15:
4
20:12:243:12:
2
10:6:249:6:
3
15:9:246:9:
3
15:9:246:9:
2
10:6:249:6:
4
20:12:243:12:
6
30:18:237:18:
8
40:24:231:24:
11
55:33:222:33:
12
60:36:219:36:
13
65:39:216:39:
12
60:36:219:36:
8
40:24:231:24:
5
25:15:240:15:
2
10:6:249:6:
3
15:9:246:9:
3
15:9:246:9:
5
25:15:240:15:
9
45:27:228:27:
11
55:33:222:33:
12
60:36:219:36:
12
60:36:219:36:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
12
60:36:219:36:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13
65:39:216:39:
13

KeyboardInterrupt: 

In [12]:
class ROVBrain:
    def __init__(self,port,baud,outs,sense):
        self.__port= port
        self.__baud= baud
        self.__outs=outs
        self.__sens=sense
        self.__outVal= [None]*len(outs)
        self.__senVal= [None]*len(sense)
        self.__ser= serial.Serial(port,baud)
        self.writeToSer(self.getInitString())
        time.sleep(0.4)
        self.__ser.flush()
        time.sleep(0.1)
        print(self.__ser.name);
        
    def writeToSer(self,string):
        self.__ser.write(string.encode('utf-8'))
    
    def readFromSer(self):
        return self.__ser.readline()
        
        
    def setLoop(self,loop):
        self.__loop=loop
        
    def run(self):
        for i in range(100000):
            self.setSensVals()
            self.__loop(self)
#             print(self.getOutsString())
            self.writeToSer(self.getOutsString())
            read=self.__ser.readline()
        
        
#         for i in range(10000000):
            
    def setSensVals(self):
#         self.__ser.flushInput()
#         self.__ser.flushOutput()
#         self.__ser.write(b'req')
        self.writeToSer("req")
        vals=self.__ser.readline()
#       print(vals) 
        sep=str(vals)[5:].split(':')
#         print(sep)
        for i in range(len(self.__sens)):
            self.__senVal[i]=int(sep[i])
        
    
    def getOutInitString(self):
        Init="set"
        for out in self.__outs:
            Init+=out.getSetString()+":"
        return Init
    
    def getSensInitString(self):
        Init="ses"
        for sens in self.__sens:
            Init+=sens.getSetString()+":"
        return Init
    
    def getInitString(self):
        return self.getSensInitString()+"n"+self.getOutInitString()#+"nret"
    
    def getOutsIndex(self,out):
        return self.__outs.index(out)
    
    def getSensIndex(self,sen):
        return self.__sens.index(sen)
    
    def getCompIndex(self,comp):
        if comp.type=="outs":
            return self.getOutsIndex();
        elif comp.type=="sens":
            return self.getSensIndex();
        else:
            print("Unknown type")
            return None
    
    def setOutsValue(self,out,value):
        self.__outVal[self.getOutsIndex(out)]=value
    
    def getOutsValue(self,out):
        return self.__outVal[self.getOutsIndex(out)]
    
    def getSensValue(self,sen):
        return self.__senVal[self.getSensIndex(sen)]
        
    def getOutsString(self):
        Outs=""
        for out in self.__outVal:
            if out!= None:
                Outs+=str(out)
            Outs+=":"
        return Outs
        
class Servo:
    def __init__(self,pin):
        self.__pin=pin
        self.__type="outs"
    
    def getSetString(self):
        return "m"+str(self.__pin)
        
    def getPin(self):
        return self.__pin
    
    def setAngle(self,brain,ang):
        brain.setOutsValue(self,ang)
        
    def getAngle(self,brain):
        return brain.__getOutsValue(self)
    
class LED:
    def __init__(self,pin):
        self.__pin=pin
        self.__type="outs"
    
    def getSetString(self):
        return "l"+str(self.__pin)
        
    def getPin(self):
        return self.__pin
    
    def setBrightness(self,brain,bright):
        brain.setOutsValue(self,bright)
        
    def getBrightness(self,brain):
        return brain.getOutsValue(self)
        
class UltraSound:
    def __init__(self,trigPin,echoPin):
        self.__trigPin=trigPin
        self.__echoPin=echoPin
        self.__type="sens"
    
    def getSetString(self):
        return "u"+str(self.__trigPin)+","+str(self.__echoPin)
        
    def getDistance(self,brain):
        return brain.getSensValue(self)    

In [24]:
Outs=[]
Sens=[]

servo1=Servo(2)
servo2=Servo(3)
servo3=Servo(4)
servo4=Servo(5)
# led1=LED(9)
# led2=LED(10)
# led3=LED(11)
Outs.append(servo1)
Outs.append(servo2)
Outs.append(servo3)
Outs.append(servo4)
# Outs.append(led1)
# Outs.append(led2)
# Outs.append(led3)

# ultra=UltraSound(5,6)
# Sens.append(ultra)
Brain=ROVBrain("/dev/cu.usbserial-1410",9600,Outs,Sens)
Brain.clock=0
def Loop(brain):
    brain.clock+=0.01
#     print(brain.clock)
    servo1.setAngle(brain,(math.sin(brain.clock)+1)*45)
    servo2.setAngle(brain,(math.sin(brain.clock)+1)*45)
    servo3.setAngle(brain,(math.sin(brain.clock)+1)*45)
    servo4.setAngle(brain,(math.sin(brain.clock)+1)*45)
    
#     dist=ultra.getDistance(brain)
#     servo.setAngle(brain,(dist*6))
#     led1.setBrightness(brain,min(dist*3,255));
#     led2.setBrightness(brain,255-min(dist*3,255));
#     led3.setBrightness(brain,min(dist*3,255));

Brain.setLoop(Loop)

Brain.run()
    

/dev/cu.usbserial-1410
0.01
0.02
0.03
0.04
0.05
0.060000000000000005
0.07
0.08
0.09
0.09999999999999999
0.10999999999999999
0.11999999999999998
0.12999999999999998
0.13999999999999999
0.15
0.16
0.17
0.18000000000000002
0.19000000000000003
0.20000000000000004
0.21000000000000005
0.22000000000000006
0.23000000000000007
0.24000000000000007
0.25000000000000006
0.26000000000000006
0.2700000000000001
0.2800000000000001
0.2900000000000001
0.3000000000000001
0.3100000000000001
0.3200000000000001
0.3300000000000001
0.34000000000000014
0.35000000000000014
0.36000000000000015
0.37000000000000016
0.38000000000000017
0.3900000000000002
0.4000000000000002
0.4100000000000002
0.4200000000000002
0.4300000000000002
0.4400000000000002
0.45000000000000023
0.46000000000000024
0.47000000000000025
0.48000000000000026
0.49000000000000027
0.5000000000000002
0.5100000000000002
0.5200000000000002
0.5300000000000002
0.5400000000000003
0.5500000000000003
0.5600000000000003
0.5700000000000003
0.5800000000000003
0.5

KeyboardInterrupt: 

In [3]:
print(serial.__file__)

/Users/martinlu/opt/anaconda3/lib/python3.7/site-packages/serial/__init__.py


In [16]:
import math